# Mount google drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Inspect Data

In [0]:
cd /content/gdrive/My\ Drive/Mizzou/Artificial\ Intelligence/AI2_final_project

/content/gdrive/My Drive/Mizzou/Artificial Intelligence/AI2_final_project


In [0]:
!head SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [0]:
lines = None
with open("SMSSpamCollection","r") as file:
  lines = file.readlines()
  
lines[0]

'ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n'

# Import libraries

In [0]:
import numpy as np
import pickle
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# Separate targets and features 

In [0]:
targets = []
features = []

with open("SMSSpamCollection","r") as file:
  for line in file:
    data = line.strip().split('\t')
    if len(data) == 2:
      t, f = data
      targets.append(t)
      features.append(f)

**Store Dataset (optional)**

In [0]:
dataset = {"features":features,"targets":targets}
pickling_on = open("dataset.pickle","wb")
pickle.dump(dataset, pickling_on)
pickling_on.close()

# Encode target labels

In [0]:
# Load the dataset
pickle_off = open("dataset.pickle","rb")
dataset = pickle.load(pickle_off)
pickle_off.close()
targets = dataset["targets"]

# Encode target labels using sklearn LabelEncoder
le = LabelEncoder()
le.fit(targets)
labels = le.transform(targets)
le.classes_

array(['ham', 'spam'], dtype='<U4')

**Save LabelEncoder (optional)**

In [0]:
pickle.dump(le, open('label_encoder.pickle','wb'))

# Split dataset into train and test (8:2) stratified



In [0]:
features = dataset["features"]
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, stratify=labels, random_state=42)

In [0]:
np.unique(y_test,return_counts=True)

(array([0, 1]), array([966, 149]))

In [0]:
np.unique(y_train,return_counts=True)

(array([0, 1]), array([3861,  598]))

**Save the train and test datasets to pickle files (optional)**

In [0]:
with open('X_train.pickle','wb') as file:
  pickle.dump(X_train,file)
  
with open('X_test.pickle','wb') as file:
  pickle.dump(X_test,file)

with open('y_train.np','wb') as file:
  np.save(file, y_train)

with open('y_test.np','wb') as file:
  np.save(file, y_test)

# Tokenize training text
 token_pattern=’(?u)\b\w\w+\b’
 
Text preprocessing, **tokenizing** and **filtering of stopwords** are all included in **CountVectorizer**, which builds a dictionary of features and transforms documents to feature vectors:

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape # sparse matrix

(4459, 7716)

**Save the CountVectorizer**

In [0]:
with open('count_vectorizer','wb') as file:
  pickle.dump(count_vect, file)

# From occurences to frequences (TF-IDF Transformation)

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tf.shape

(4459, 7716)

In [0]:
X_train_tf[0]

<1x7716 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

**Save the tfidf_transformer (checkpoint save)**

In [0]:
with open("tfidf_transformer","wb") as file:
  pickle.dump(tfidf_transformer, file)

# Grid Search to optimize SVM(later?)
[Grid Search](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html)

Load preprocessing models and training dataset 

In [0]:
encoder = pickle.load(open('label_encoder.pickle','rb'))
count_vect = pickle.load(open('count_vectorizer','rb'))
tfidf_transformer = pickle.load(open("tfidf_transformer","rb"))

In [0]:
X_train = pickle.load(open('X_train.pickle','rb'))
y_train = np.load(open('y_train.np','rb'))
X_test = pickle.load(open('X_test.pickle','rb'))
y_test = np.load(open('y_test.np','rb'))

## SVC parameter tuning

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import matthews_corrcoef, precision_score, recall_score, accuracy_score

import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

# Classifier
svc = SVC()

# Build a pipeline
pipe = Pipeline(steps=[('vect', count_vect),
                       ('tfidf', tfidf_transformer),
                       ('svc', svc)])

# Set the parameters by cross-validation
tuned_params = [{'svc__kernel': ['rbf'], 'svc__gamma': [1e-3, 1e-4], 'svc__C': [1, 10, 100, 1000]},
                {'svc__kernel': ['linear'], 'svc__C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall', 'balanced_accuracy']

for score in scores:
  print("# Tuning hyper-parameters for %s" % score)
  print()
    
  search = GridSearchCV(pipe, tuned_params, cv=5, scoring=score)
  
  search.fit(X_train, y_train)
  
  print("Best parameters set found on development set:")
  print()
  print(search.best_params_)
  print()
  print("Grid scores on development set:")
  print()
  means = search.cv_results_['mean_test_score']
  stds = search.cv_results_['std_test_score']
  
  for mean, std, params in zip(means, stds, search.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
  print()
  
  print("Detailed classification report:")
  print()
  print("The model is trained on the full development set.")
  print("The scores are computed on the full evaluation set.")
  print()
  y_true, y_pred = y_test, search.predict(X_test)
  print(classification_report(y_true, y_pred))
  print()
  print("Spam Caught: %0.02f %%" % (100*recall_score(y_true, y_pred)))
  print("Blocked Ham: %0.02f %%" % (100*(1-precision_score(y_true, y_pred))))
  print("Accuracy: %0.02f %%" % (100*accuracy_score(y_true, y_pred)))
  print("Matthews correlation coefficient (MCC): %0.02f %%" %(100*matthews_corrcoef(y_true, y_pred)))
    


# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'svc__C': 100, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}

Grid scores on development set:

0.000 (+/-0.000) for {'svc__C': 1, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
0.000 (+/-0.000) for {'svc__C': 1, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf'}
0.000 (+/-0.000) for {'svc__C': 10, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
0.000 (+/-0.000) for {'svc__C': 10, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf'}
0.993 (+/-0.011) for {'svc__C': 100, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
0.000 (+/-0.000) for {'svc__C': 100, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf'}
0.984 (+/-0.033) for {'svc__C': 1000, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
0.993 (+/-0.011) for {'svc__C': 1000, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf'}
0.986 (+/-0.031) for {'svc__C': 1, 'svc__kernel': 'linear'}
0.979 (+/-0.032) for {'svc__C': 10, 'svc__kernel': 'linear'}
0.979 (+/-0.032) for {'svc__C': 100, 'svc__kernel': 'linea

# Performance metrics for different optimization

**Precision-optimized model**

Spam Caught: 83.89 %

Blocked Ham: 0.00 %

Accuracy: 97.85 %

Matthews correlation coefficient (MCC): 90.48 %


**Recall-optimized model**

Spam Caught: 92.62 %

Blocked Ham: 2.13 %

Accuracy: 98.74 %

Matthews correlation coefficient (MCC): 94.50 %

**Accuracy-optimized model**

Spam Caught: 92.62 %

Blocked Ham: 2.13 %

Accuracy: 98.74 %
Matthews correlation coefficient (MCC): 94.50 %

**Performance metrics terms**

Blocked Ham: FP/(TP+FP) =  1 - Precision

Spam Caught: TP/(TP+FN) =  Recall

Accuracy: Acc

Matthews correlation coefficient: MCC